In [ ]:
import os, osgeo
# del os.environ['PROJ_LIB']
import math
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
from pyproj import proj
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum
import rasterio
from rasterstats import zonal_stats
import xarray as xr
import numpy as np
from osgeo import gdal, gdalconst, osr
import sys, time, ogr
import libpysal
gdal.UseExceptions()  # not required, but a good idea
gdal.AllRegister()

In [ ]:
wv3src = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2021-WorldView3-8BAnds\21MAY13074013-S2AS-RCRC-8Bands.TIF"
wv4dst= r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Riyadh\2021-WorldView3-8BAnds\21MAY13074013-S2AS-RCRC-8Bands-SAVI.TIF"
src=rasterio.open(wv3src)


NameError: name 'rasterio' is not defined

In [2]:
def process_NDVI_image(src_img, dst_img, nir_band_id=8,red_band_id=5):
    with rasterio.open(src_img) as src:
        kwargs = src.meta
        tiles = src.block_windows(red_band_id)
        profile = src.profile
        # print(profile)
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw',
            BIGTIFF = "IF_SAFER")

        with rasterio.open(dst_img, 'w', **profile) as dst:
            for idx, window in tiles:
                print("Processing Block: ", idx[0]+1, ", ", idx[1]+1)
                np.seterr(divide='ignore', invalid='ignore')
                # ndval = src.nodatavals[0]
                # print(ndval)
                red_data = src.read(red_band_id, window=window)

                # red_data[red_data==ndval] = np.nan

                nir_data = src.read(nir_band_id, window=window)
                # nir_data[nir_data==ndval] = np.nan
                ndvi_con=np.where(
                    (nir_data.astype(float)+red_data.astype(float))==0., #It will return 0 for No Data value
                    0,
                    (nir_data.astype(float) - red_data.astype(float)) / (nir_data.astype(float) + red_data.astype(float)))
                # ndvi_con = (nir_data - red_data)/(nir_data + red_data)
                L = 0.5
                savi_con=np.where(
                    (nir_data.astype(float)+red_data.astype(float))==0., #It will return 0 for No Data value
                    0,
                    (nir_data.astype(float) - red_data.astype(float)) / (nir_data.astype(float) + red_data.astype(float) + L) * (1 + L))

                dst.write_band(1, savi_con, window=window)
                dst.write_band(1, ndvi_con, window=window)

    return 0

In [ ]:
process_NDVI_image(wv3src,wv4dst)